In [25]:
#Loading libraries 
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler 

In [2]:
data = pd.read_csv('data/processed_diabetes_data.csv')

In [3]:
class0 = data[data["readmitted"] == 0]
class1 = data[data["readmitted"] == 1]
majorityCount = class0.shape[0]
minorityCount = class1.shape[0]
class01, class02 = class0.sample(frac=0.1, random_state=42), class0.sample(frac=0.9, random_state=42)
print("Class 0 (readmitted = 0): ", majorityCount)
print("Class 1 (readmitted = 1): ", minorityCount)
print("20% Resampled Class 0 (readmitted = 0):", class01.shape[0])
print("Class 1 (readmitted= 1): ", minorityCount)

oversampled_class1 = class1.sample(frac=0.1, replace=True, random_state=42)
sampled_df = pd.concat([class01, oversampled_class1])
sampled_df.drop(columns=["Unnamed: 0"]).describe()

Class 0 (readmitted = 0):  90406
Class 1 (readmitted = 1):  11357
20% Resampled Class 0 (readmitted = 0): 9041
Class 1 (readmitted= 1):  11357


,age,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmitted,...,examide,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed
count,10177.000000,10177.000000,10177.000000,10177.000000,10177.000000,10177.000000,10177.000000,10177.000000,10177.000000,10177.000000,...,10177.0,10177.0,10177.000000,10177.000000,10177.000000,10177.0,10177.0,10177.0,10177.000000,10177.000000
mean,65.985556,4.367004,43.134126,1.331041,16.036651,0.364253,0.192002,0.631424,7.395991,0.111624,...,0.0,0.0,1.415054,1.007271,0.000098,0.0,0.0,0.0,0.531984,0.770954
std,15.900063,2.943190,19.474766,1.723683,8.120029,1.229179,0.720927,1.253708,1.943385,0.314919,...,0.0,0.0,0.846098,0.086114,0.009913,0.0,0.0,0.0,0.499000,0.420239
min,5.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,...,0.0,0.0,0.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000
25%,55.000000,2.000000,32.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000,0.000000,...,0.0,0.0,1.000000,1.000000,0.000000,0.0,0.0,0.0,0.000000,1.000000
50%,65.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000,0.000000,...,0.0,0.0,1.000000,1.000000,0.000000,0.0,0.0,0.0,1.000000,1.000000
75%,75.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000,0.000000,...,0.0,0.0,2.000000,1.000000,0.000000,0.0,0.0,0.0,1.000000,1.000000
max,95.000000,14.000000,102.000000,6.000000,69.000000,38.000000,12.000000,16.000000,16.000000,1.000000,...,0.0,0.0,3.000000,3.000000,1.000000,0.0,0.0,0.0,1.000000,1.000000


In [4]:
data = sampled_df

In [5]:
X = data.drop('readmitted',axis=1)

y = data['readmitted']

train, test = train_test_split(data, test_size=0.8, random_state=42)
print('Train:', train.shape[0])
print('Test:', test.shape[0])

Train: 2035
Test: 8142


In [6]:
#all LGBM features

In [17]:
import lightgbm as lgb
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler

X = data.drop('readmitted',axis=1)
y = data['readmitted']

# Define and fit a StandardScaler
scaler = StandardScaler()
model = scaler.fit(X)
scaled_X = model.transform(X)

# Split data into train and test sets

X_train, X_test, y_train, y_test = train_test_split(scaled_X, y, test_size=0.2, random_state=42, stratify=y)

# Define LightGBM parameters
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    'num_leaves': [31, 50, 100],
}

# Create and fit a LightGBM model
lgb_classifier = lgb.LGBMClassifier()
lgb_classifier.fit(X_train, y_train)

grid_search = GridSearchCV(lgb_classifier, param_grid, cv=5, scoring='f1')
grid_search.fit(X_train, y_train)

# Get the best hyperparameters
best_params = grid_search.best_params_
best_estimator = grid_search.best_estimator_

print("Best Hyperparameters:", best_params)

Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 300, 'num_leaves': 50}


In [23]:
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score

# Create and fit a LightGBM model with the selected attributes
params= {'learning_rate': 0.01,
 'max_depth': 3,
 'n_estimators': 100,
 'num_leaves': 31}

model_selected_attributes = lgb.LGBMClassifier(**params)
model_selected_attributes.fit(X_train, y_train)

# Predict on the test set
y_pred = best_estimator.predict(X_test)

# Evaluate the model with selected attributes
accuracy_selected_attributes = accuracy_score(y_test, y_pred)
roc_auc_selected_attributes = roc_auc_score(y_test, model_selected_attributes.predict_proba(X_test)[:, 1])
precision_selected_attributes = precision_score(y_test, y_pred)
recall_selected_attributes = recall_score(y_test, y_pred)
f1_selected_attributes = f1_score(y_test, y_pred)

# Compute confusion matrix for the model with selected attributes
confusion_matrix_selected_attributes = confusion_matrix(y_test, y_pred)

print("\nModel with selected attributes:")
print("Accuracy:", accuracy_selected_attributes)
print("ROC AUC:", roc_auc_selected_attributes)
print("Precision:", precision_selected_attributes)
print("Recall:", recall_selected_attributes)
print("F1 Score:", f1_selected_attributes)
cm = confusion_matrix_selected_attributes
print("Confusion Matrix:\n", cm)



Model with selected attributes:
Accuracy: 0.8899803536345776
ROC AUC: 0.6645894365665553
Precision: 0.5272727272727272
Recall: 0.1277533039647577
F1 Score: 0.20567375886524822
Confusion Matrix:
 [[1783   26]
 [ 198   29]]
